In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [40, 30]
import numpy as np
import dill as pickle

from src.data.dataset import *
from src.data import loop_ast


In [4]:
d= DatasetFromPkl('data/speedup_dataset.pkl')

In [14]:
np.all(np.array((1, 1, 0)) + np.array((-1, 1, 1)) != 1)

False